<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#NaN-и-явные-дубликаты" data-toc-modified-id="NaN-и-явные-дубликаты-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>NaN и явные дубликаты</a></span></li><li><span><a href="#Разделение-на-featueres-и-target" data-toc-modified-id="Разделение-на-featueres-и-target-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Разделение на featueres и target</a></span></li><li><span><a href="#Промежуточный-вывод" data-toc-modified-id="Промежуточный-вывод-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Промежуточный вывод</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Промежуточный-вывод" data-toc-modified-id="Промежуточный-вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Промежуточный вывод</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Промежуточный-вывод" data-toc-modified-id="Промежуточный-вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Промежуточный вывод</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп», разработав такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось, а так же обосновать корректность созданного алгоритма.

In [1]:
#импорт библиотек
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

Для начала, загрузим данные из предоставленного датасета.

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.info()
data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


У нас небольшой датасет: всего 5000 строк и 5 столбцов, все численные.

### NaN и явные дубликаты
Проверим наличие пропущенных значений.

In [3]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Пропусков в данных нет.Посмотрим, есть ли полные дубликаты.

In [4]:
data.duplicated().sum()

153

Дубликаты присутствуют, их мы удалим за ненадобностью. Иначе они могут повлиять на результаты исследования.

In [5]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Разделение на featueres и target

Отделим целевые признаки от всех остальных.

In [6]:
# Отделяем целевой признак
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']
print('features shape:', features.shape)
print('target shape:', target.shape)

features shape: (4847, 4)
target shape: (4847,)


Целевой признак "Страховые выплаты" отделён от всех остальных в обеих выборках.

### Промежуточный вывод

В пункте 1 "Загрузка данных" мы загрузили и изучили датасет, провели необходимую предобработку. Теперь данные готовы к дальнейшей работе над ними.

## Умножение матриц

Целью в данном проекте является составление алгоритма, который позволит зашифровать данные без потери качества линейной регрессии. Для защиты данных мы будем умножать их на матрицу.

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится.

**Обоснование:**

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Функция минимизации потерь:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения весов модели линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

В формуле обучения, матрицу признаков X умножаем на матрицу для шифрования P:

$$
w = ((XP)^T (XP))^{-1} (XP)^T y
$$

По свойствам обратной матрицы $ (AB)^{-1} = A^{-1}B^{-1} $ и транспортированной матрицы $ (AB)^T = A^TB^T $ расскроем скобки:

$$
w = ((XP)^TXP){-1}(XP)^T y
$$

Теперь применим свойство обратной матрицы $ AA^{-1} = A^{-1}A = E $, где E — это единичная матрица:

$$
w = E (X^TX)^{-1}(P^T)^{-1}P^{-1}X^TP^T y
$$

При этом при умножении E на любую матрицу, получается та же самая матрица $ AE = EA = E $, следовательно:

$$
w = (X^T)^{-1} X^{-1}P^{-1} X^T y
$$

Используя ранее обозначенную формулу для обучения весов модели $ w = (X^T X)^{-1} X^T y $ мы можем сжать наше уранение до следующего вида:

$$
w = P^{-1}w
$$

Возвращаемся к формуле предсказания $ a = Xw $. Подставим в неё $ X = XP $ и $ w = P^{-1}w $:

$$ a_p = XPP^{-1}w = XEw = Xw = a $$

### Промежуточный вывод

Поскольку веса в исходной и преобразованной задаче связаны, наличие матрицы шифрования не влияет на качество модели.

## Алгоритм преобразования

**Алгоритм**

Алгоритм преобразования простой: в методе fit() класса LinearRegression() мы будем при помощи псевдослучайных чисел создавать обратимую матрицу P размера n на n и на неё перемножать признаки X размера m на n.

Дальнейший процесс обучения будет происходить так же, как и в обычной линейной регрессии.

При необходимости дешифровки данные можно восстановить, умножив на обратную матрицу $ P^{-1} $: $$XPP^{-1} = XE = X$$

**Обоснование**

Исходя из ответа в п.2, в случае умножения признаков на матрицу для шифрования качество модели линейной регрессии измениться не должно.

## Проверка алгоритма

Будем обучать сразу две модели линейной регрессии: одну без шифрования, и одну с ним.

Для начала, сгенерируем матрицу шифрования и умножим на неё признаки.

In [7]:
rnd=np.random.RandomState(12345)
det = 0
#В цикле повторяем создание матрицы, пока она точно не будет обратимой
while det == 0:
    encryption_matrix = rnd.randint(0, 1000, size=(features.shape[1], features.shape[1]))
    det = np.linalg.det(encryption_matrix)
features_encrypted = features @ encryption_matrix
features_encrypted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4847 non-null   float64
 1   1       4847 non-null   float64
 2   2       4847 non-null   float64
 3   3       4847 non-null   float64
dtypes: float64(4)
memory usage: 151.6 KB


Теперь у нас есть зашифрованные признаки. Переходим к обучению моделей.

In [8]:
#обычная модель
model_regular = LinearRegression()
model_regular.fit(features, target)
predictions_regular = model_regular.predict(features)
r2_regular = r2_score(target, predictions_regular)

#модель с шифрованием
model_encrypted = LinearRegression()
model_encrypted.fit(features_encrypted, target)
predictions_encrypted = model_encrypted.predict(features_encrypted)
r2_encrypted = r2_score(target, predictions_encrypted)

print('R2 метрика без шифрования:', r2_regular.round(5))
print('R2 метрика с шифрованием:', r2_encrypted.round(5))
print('Разница:', (r2_regular - r2_encrypted).round(5))

R2 метрика без шифрования: 0.4302
R2 метрика с шифрованием: 0.4302
Разница: 0.0


### Промежуточный вывод

Как и ожидалось, разницы в качестве модели с шифрованием и без нет, а это значит, данный метод можно использовать для защиты информации без опаски неработоспособности модели.

## Вывод

В данном проекте задачей было разработать алгоритм шифрования данных о клиентах страховой компании без потери качества работы модели линейной регрессии. С этой целью были проведены следующие работы:

- Загружены, изучены и предобработаны данные для обучения;
- Доказано, что шифрование не влияет на качество модели;
- Разработан алгоритм преобразования данных при помощи матрицы шифрования;
- Проверена работоспособность алгоритма.

Мною было подтверждено, что умножение признаков (данных клиентов) на матрицу шифрования не влияет на качество модели линейной регрессии, а значит, такой способ можно использовать для защиты личной информации клиентов компании, и при этом иметь возможность проводить статистические исследования.